# DSE Course 1, Session 6: Product Margin Case Study

**Instructor**: Wesley Beckner

**Contact**: wesleybeckner@gmail.com

<br>

---

<br>

In this session we will look at how EDA and statistical analysis can allow us to ask "what if" questions around a manufacturing product portfolio

EDA objectives:

* product elimination impact on annual margin
* evaluating statistical significance of product margin

<br>

---


<a name='x.0'></a>

## 6.0 Preparing Environment and Importing Data

[back to top](#top)

<a name='x.0.1'></a>

### 6.0.1 Import Packages

[back to top](#top)

In [ ]:
# Pandas library for the pandas dataframes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import random
import scipy.stats

<a name='x.0.2'></a>

### 6.0.2 Load Dataset

[back to top](#top)

For this session, we will use dummy datasets from sklearn.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/wesleybeckner/'\
                 'ds_for_engineers/main/data/truffle_margin/margin_data.csv')
df['Width'] = df['Width'].apply(str)
df['Height'] = df['Height'].apply(str)

In [ ]:
descriptors = df.columns[:-3]

## 6.1 Many Flavors of Statistical Tests

<p align="center">
<img src="https://luminousmen.com/media/descriptive-and-inferential-statistics.jpeg" width=400px></img>
<small> https://luminousmen.com/post/descriptive-and-inferential-statistics </small>
</p>

>Descriptive statistics describes data (for example, a chart or graph) and inferential statistics allows you to make predictions (“inferences”) from that data. With inferential statistics, you take data from samples and make generalizations about a population - [statshowto](https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/inferential-statistics/#:~:text=Descriptive%20statistics%20describes%20data%20(for,make%20generalizations%20about%20a%20population.)

* **Moods Median Test**
* [Kruskal-Wallis Test](https://sixsigmastudyguide.com/kruskal-wallis-non-parametric-hypothesis-test/)
* T-Test
* Analysis of Variance (ANOVA)
  * One Way ANOVA
  * Two Way ANOVA
  * MANOVA
  * Factorial ANOVA

When do I use each of these? We will talk about this as we proceed through the examples. [This page](https://support.minitab.com/en-us/minitab/20/help-and-how-to/statistics/nonparametrics/supporting-topics/which-test-should-i-use/) from minitab has good rules of thumb on the subject.



### 6.1.1 What is Mood's Median?

**A special case of Pearon's Chi-Squared Test:** We create a table that counts the observations above and below the global median for two different groups. We then perform a *chi-squared test of significance* on this *contingency table* 

Null hypothesis: the Medians are all equal

The chi-square test statistic:

$x^2 = \sum{\frac{(O-E)^2}{E}}$

Where $O$ is the observed frequency and $E$ is the expected frequency.

**Let's take an example**, say we have three shifts with the following production rates:

In [ ]:
random.seed(42)
def my_little_generator(mean, noise):
  return round(mean + noise * random.random() * random.choice([-1, 1]))

In [ ]:
shift_one = [my_little_generator(16, 4) for i in range(10)]
shift_two = [my_little_generator(19, 4) for i in range(10)]

In [ ]:
print(shift_one)
print(shift_two)

[13, 13, 15, 13, 14, 16, 15, 18, 17, 13]
[16, 22, 18, 23, 19, 19, 17, 16, 23, 19]


In [ ]:
stat, p, m, table = scipy.stats.median_test(shift_one, shift_two, correction=False)

what is `median_test` returning?

In [ ]:
print("The perasons chi-square test statistic: {:.2f}".format(stat))
print("p-value of the test: {:.3f}".format(p))
print("the grand median: {}".format(m))

The perasons chi-square test statistic: 7.20
p-value of the test: 0.007
the grand median: 16.5


Let's evaluate that test statistic ourselves by taking a look at the contingency table:

In [ ]:
table

array([[2, 8],
       [8, 2]])

This is easier to make sense of if we order the shift times

In [ ]:
shift_one.sort()
shift_one

[13, 13, 13, 13, 14, 15, 15, 16, 17, 18]

When we look at shift one, we see that 8 values are at or below the grand median.

In [ ]:
shift_two.sort()
shift_two

[16, 16, 17, 18, 19, 19, 19, 22, 23, 23]

For shift two, only two are at or below the grand median.

Since the sample sizes are the same, the expected value for both groups is the same, 5 above and 5 below the grand median. The chi-square is then:

$X^2 = \frac{(2-5)^2}{5} + \frac{(8-5)^2}{5} + \frac{(8-5)^2}{5} + \frac{(2-5)^2}{5}$


In [ ]:
(2-5)**2/5 + (8-5)**2/5 + (8-5)**2/5 + (2-5)**2/5

7.2

Our p-value, or the probability of observing the null-hypothsis, is under 0.05. We can conclude that these shift performances were drawn under seperate distributions.

### 6.1.2 When to Use Mood's?

**Mood's Median Test is highly flexible** but has the following assumptions:

* Considers only one categorical factor
* Response variable is continuous
* Data does not need to be normally distributed
  * But the distributions are similarly shaped
* Sample sizes can be equal and small (less than 20 observations)

Other considerations:

* Not as powerful as Kruskal-Wallis Test but still useful for small sample sizes or when there are outliers

### 6.1.3 What is a T-test?

A T-test has the same objective as the moods median test, but operates slightly differently, and on the means rather than medians of the data. Let's have a look

Recall:

In [ ]:
print(shift_one)
print(shift_two)

[13, 13, 13, 13, 14, 15, 15, 16, 17, 18]
[16, 16, 17, 18, 19, 19, 19, 22, 23, 23]


To calculate the T-test, we follow a slightly different statistical formula:

$T=\frac{\mu_1 - \mu_2}{s\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}$

where $\mu$ are the means of the two groups, $n$ are the sample sizes and $s$ is the pooled standard deviation, also known as the cummulative variance (depending on if you square it or not):

$s= \sqrt{\frac{(n_1-1)\sigma_1^2 + (n_2-1)\sigma_2^2}{n_1 + n_2 - 2}}$

where $\sigma$ are the standard deviations. What you'll notice here is we are combining the two variances, we can only do this if we assume the variances are somewhat equal, this is known as the *equal variances* t-test.

In [ ]:
mean_shift_one = np.mean(shift_one)
mean_shift_two = np.mean(shift_two)

print(mean_shift_one, mean_shift_two)

14.7 19.2


In [ ]:
com_var = ((np.sum([(i - mean_shift_one)**2 for i in shift_one]) + 
            np.sum([(i - mean_shift_two)**2 for i in shift_two])) /
            (len(shift_one) + len(shift_two)-2))
print(com_var)

5.205555555555555


In [ ]:
T = (np.abs(mean_shift_one - mean_shift_two) / (
     np.sqrt(com_var/len(shift_one) +
     com_var/len(shift_two))))

In [ ]:
T

4.410257762597409

We see that this hand-computed result matches that of the `scipy` module:

In [ ]:
scipy.stats.ttest_ind(shift_two, shift_one, equal_var=True)

Ttest_indResult(statistic=4.410257762597408, pvalue=0.00033771529900370443)

There are actually three flavors of independent, two-sample t-tests:

* Euqal sample sizes and variance
* Equal variance (or pooled) t-test
  * `scipy.stats.ttest_ind(equal_var=True)`
* Unequal variance t-test
  * `scipy.stats.ttest_ind(equal_var=False)`

<br>

We also have dependent t-tests:
* Paired (or correlated) t-test
  * `scipy.stats.ttest_rel`

A full discussion on t-tests is outside the scope of this session, but we can refer to wikipedia for more information, including formulas on how each statistic is computed:
* [student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test#Dependent_t-test_for_paired_samples)

### 6.1.4 What is Analysis of Variance? 

* Analysis of Variance (ANOVA)
  * extension of a t-test, is there a difference of means between the *groups*?
  * A t-test only tests two groups at a time
  * a t-test compares means, ANOVA compares variances
* Looks for
  * The variation within groups
  * The variation between groups

The algorithm for ANOVA is a bit more complicated, so we won't compute this by hand. In the next session, we will explore ANOVA on our wine dataset.

In summary, there are many statistical tests at our disposal when performing inferential statistical analysis. In times like these, a simple decision tree can be extraordinarily useful!

<img src="https://cdn.scribbr.com/wp-content/uploads//2020/01/flowchart-for-choosing-a-statistical-test.png" width=800px></img>

<small>source: [scribbr](https://www.scribbr.com/statistics/statistical-tests/)</small>

## 6.2 Evaluate statistical significance of product margin

### 6.2.1 Mood's Median on product descriptors

The first issue we run into with moods is... what? 

We can only perform moods on two groups at a time. How can we get around this?

In [ ]:
df.columns

Index(['Base Cake', 'Truffle Type', 'Primary Flavor', 'Secondary Flavor',
       'Color Group', 'Width', 'Height', 'Net Sales Quantity in KG', 'EBITDA',
       'Product'],
      dtype='object')

In [ ]:
delimiters = df.columns[:-3]
moodsdf = pd.DataFrame()
pop = list(df['EBITDA'])
# pop = np.random.choice(pop, size=int(1e5))
for delimiter in delimiters:
    grouped = df.groupby(delimiter)['EBITDA']
    group_with_values = grouped.apply(list)

    # bootstrap population of values based on groups
#     pop = np.random.choice((np.concatenate(group_with_values)), 
#                            size=int(1e4))
    
    for index, group in enumerate(group_with_values):
        stat, p, m, table = scipy.stats.median_test(group, pop)
        median = np.median(group)
        mean = np.mean(group)
        size = len(group)
        moodsdf = pd.concat([moodsdf, 
                                 pd.DataFrame([delimiter, 
                                               group_with_values.index[index],
                                               stat, p, m, mean, median, size, 
                                               table]).T])
moodsdf.columns = ['descriptor', 'group', 'pearsons_chi_square', 'p_value', 
                   'grand_median', 'group_mean', 'group_median', 'size', 
                   'table']


In [ ]:
moodsdf = moodsdf.loc[moodsdf['p_value'] < 1e-3]
moodsdf = moodsdf.sort_values('group_median').reset_index(drop=True)

In [ ]:
moodsdf

,descriptor,group,pearsons_chi_square,p_value,grand_median,group_mean,group_median,size,table
0,Secondary Flavor,Cucumber,12.5898,0.000387861,22.05,-18454.5,-7756.69,18,"[[1, 1261], [17, 1245]]"
1,Primary Flavor,Orange,12.5898,0.000387861,22.05,-18454.5,-7756.69,18,"[[1, 1261], [17, 1245]]"
2,Truffle Type,Jelly Filled,12.5898,0.000387861,22.05,-18454.5,-7756.69,18,"[[1, 1261], [17, 1245]]"
3,Primary Flavor,Creme de Menthe,17.535,2.82072e-05,18.49,-9320.5,-5945.59,23,"[[1, 1263], [22, 1243]]"
4,Secondary Flavor,Papaya,90.8672,1.53648e-21,-27.62,-3790.46,-1683.78,115,"[[7, 1303], [108, 1203]]"
5,Primary Flavor,Orange Pineapple\tP,90.8672,1.53648e-21,-27.62,-3790.46,-1683.78,115,"[[7, 1303], [108, 1203]]"
6,Secondary Flavor,Peppermint,103.973,2.05086e-24,-46.96,-4890.25,-1580.45,157,"[[16, 1315], [141, 1191]]"
7,Primary Flavor,Cream Soda,103.973,2.05086e-24,-46.96,-4890.25,-1580.45,157,"[[16, 1315], [141, 1191]]"
8,Secondary Flavor,Wild Cherry Cream,17.1878,3.38605e-05,8,-5155.42,-1434.09,69,"[[17, 1270], [52, 1236]]"
9,Primary Flavor,Lemon Bar,17.1878,3.38605e-05,8,-5155.42,-1434.09,69,"[[17, 1270], [52, 1236]]"


## 6.3 product elimination impact on annual margin